In [29]:
import numpy as np
import pandas as pd

import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader

import gluonnlp as nlp

#kobert
from kobert.utils import get_tokenizer
from kobert.pytorch_kobert import get_pytorch_kobert_model

#GPU 사용
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

#BERT 모델, Vocabulary 불러오기
bertmodel, vocab = get_pytorch_kobert_model()

using cached model
using cached model


In [15]:
class BERTDataset(Dataset):
    def __init__(self, dataset, bert_tokenizer, args):
        transform = nlp.data.BERTSentenceTransform(
            bert_tokenizer, max_seq_length=args.max_len, pad=args.pad, pair=args.pair)

        self.sentences = [transform([i]) for i in dataset]

    def __getitem__(self, i):
        return self.sentences[i]

    def __len__(self):
        return (len(self.sentences))

In [16]:
class BERTClassifier(nn.Module):
    def __init__(self, 
                 bert, 
                 hidden_size=768, 
                 num_classes=7,
                 dr_rate=0.5,
                 params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate
                 
        self.classifier = nn.Linear(hidden_size, num_classes)
        if self.dr_rate:
            self.dropout = nn.Dropout(p=self.dr_rate)
    
    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)
        
        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device))
        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out)

In [51]:
def test(model, data_test, args):
    test_dataloader = DataLoader(data_test, 
                                 batch_size=args.batch_size) #, num_workers=args.num_workers) 

    model.eval()
    
    current_preds = []
    
    with torch.no_grad():
        for token_ids, valid_length, segment_ids in test_dataloader:
            token_ids = token_ids.long().to(device)
            segment_ids = segment_ids.long().to(device)
            valid_length= valid_length
            out = model(token_ids, valid_length, segment_ids)
            
            _, predicted = torch.max(out.data, 1)

            current_preds.extend(predicted)
            
    return current_preds

In [17]:
model = BERTClassifier(bertmodel).to(device)
model.load_state_dict(torch.load('./weight/bert_1.pt'))

<All keys matched successfully>

In [18]:
import argparse
from copy import deepcopy

#토큰화
tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)

# ====== Random seed Initialization ====== #
seed = 42
np.random.seed(seed)
torch.manual_seed(seed)

parser = argparse.ArgumentParser()
args = parser.parse_args("")

# Setting parameters
args.sent_idx = 0
args.label_idx = 1
args.max_len = 64
args.pad = True
args.pair = False

args.num_workers = 4
args.batch_size = 64

args.warmup_ratio = 0.1
args.num_epochs = 20

args.max_grad_norm = 1

args.log_interval = 100

args.learning_rate =  5e-5

using cached model


In [11]:
data_list = []

In [100]:
question = input('문장을 입력하세요 :')
data_list.append(question)

In [101]:
data_list

['창가로 옮겼더니 너무 춥다..',
 '아직도 창밖은 별빛으로 가득해 !!',
 '귀여운건 거꾸로 해도 귀엽지',
 '긴 문장에 대한 효용성은 아무리 강조해도 지나치지 않지만 그것이 어떠한 감정을 설명한 다는 것은 무척이나 빈약한 근거 위에 쌓아올린 논리라고 생각해',
 '오오오오오오',
 '이럴수가',
 '우오오오오오오오오오오와',
 '배고픈데 치과가기 싫어',
 '크리스마스트리만들었다!',
 '이거 사이코패스한테 팔자',
 '오늘 학원등록할게요',
 '아티제 화이트롤이랑 아이스 아메리카노를 먹으러 갈거예요',
 '크리스마스캐롤을 틀어놓고 나는 샤워를 합니다',
 '짜장면에 탕수육 소스는 부을까요 말까요']

In [102]:
data_test = BERTDataset(data_list, tok, args)

In [103]:
for i in data_test:
    print(i)

(array([   2, 4438, 5330, 6079,  517, 6976, 5422, 5838, 1458,  517, 7470,
       5782,  517,   54,  517,   54,    3,    1,    1,    1,    1,    1,
          1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
          1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
          1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
          1,    1,    1,    1,    1,    1,    1,    1,    1]), array(17), array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]))
(array([   2, 3129, 5859, 4438, 6284, 7086, 2348, 6458, 7078,  745, 7848,
        517,    5,  517,    5,    3,    1,    1,    1,    1,    1,    1,
          1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
          1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
          1,    1,    1,    1,    1,    1,    1,    

In [104]:
pred_list = test(model, data_test, args)

In [105]:
for i, e in enumerate(pred_list):
    a = e.to('cpu').numpy()
    if a == 0:
        a = '공포'
    elif a == 1:
        a = '놀람'
    elif a == 2:
        a = '분노'
    elif a == 3:
        a = '슬픔'
    elif a == 4:
        a = '중립'
    elif a == 5:
        a = '행복'
    elif a == 6:
        a = '혐오'
    
    pred_list[i] = a
    
pred_list 

['슬픔',
 '행복',
 '행복',
 '중립',
 '중립',
 '놀람',
 '행복',
 '슬픔',
 '행복',
 '분노',
 '중립',
 '중립',
 '중립',
 '공포']

In [106]:
df = pd.DataFrame()
df['Sentrnce'] = data_list
df['Emotion'] = pred_list

In [107]:
df

,Sentrnce,Emotion
0,창가로 옮겼더니 너무 춥다..,슬픔
1,아직도 창밖은 별빛으로 가득해 !!,행복
2,귀여운건 거꾸로 해도 귀엽지,행복
3,긴 문장에 대한 효용성은 아무리 강조해도 지나치지 않지만 그것이 어떠한 감정을 설명...,중립
4,오오오오오오,중립
5,이럴수가,놀람
6,우오오오오오오오오오오와,행복
7,배고픈데 치과가기 싫어,슬픔
8,크리스마스트리만들었다!,행복
9,이거 사이코패스한테 팔자,분노
